In [2]:
import ablang2

# heavy chain sequence
heavy_ablang = ablang2.pretrained("heavy")
heavy_ablang.freeze()

AssertionError: The selected model to use (heavy) does not exist.        Please select a valid model.

In [2]:
import pandas as pd

df = pd.read_csv("./covabdab_all.csv")

# get the heavy chain sequence "seqs", the neutralising data (0 or 1) "neutralising", and PDB IDs "pdbids"
seqs = []
neutralising = []
pdbids = []

frame_size = len(df.index)

# There are some problems in some sequences,let's try the first 3000 records for now
#MAXCNT = frame_size
MAXCNT = 3000

cnt = 0
for i in range(frame_size):
    if (cnt < MAXCNT and str(df.iloc[i]['Origin']).lower().find('human') != -1 and df.iloc[i]['VHorVHH'] != 'ND'):
        seqs.append(df.iloc[i]['VHorVHH'])
        # somehow the empty string became "nan"
        if(str(df.iloc[i]['Neutralising Vs']) != "nan"):
            neutralising.append(1)
        else:
            neutralising.append(0)
            
        if(str(df.iloc[i]['Structures']).find('PDB entry') != -1):
            pdbids.append(str(df.iloc[i]['Structures'])[10:14])
        else:
            pdbids.append('')
            
        cnt = cnt + 1

print ("Lines:", len(seqs))
print (seqs[0:2])
print (neutralising)
print (pdbids)

Lines: 3000
['EVQLVQSGAEVSQPGESLKISCKGSGYSFTGYWISWVRQMPGKGLEWMGIIYPGDSDTKYTPSFQGQVTISTDKSINTAYLQWSSLKASDTAMYYCARRGDGLYYYGMDVWGQGTTVTVSS', 'EVQLVESGPGLVKPSETLSLTCTASGGSISTYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARDRIAPVGKFFGWYFDLWGRGTLVTVSS']
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
# use the "Seq-coding" example to get an array for each sequence
seqcodings = heavy_ablang(seqs, mode='seqcoding')

print("The output shape of the seq-codings:", seqcodings.shape)

print(seqcodings)

The output shape of the seq-codings: (3000, 768)
[[-0.09330826  0.64819845 -1.08619225 ... -0.59027605  0.34280308
   0.80515875]
 [-0.45837705  0.50708104 -1.15761436 ... -0.21607836  0.77288645
   1.13230815]
 [-0.39963617  0.87048231 -0.38694611 ... -0.74714346  0.69520932
   1.18203556]
 ...
 [-0.19876792 -0.24587346 -1.34513009 ... -0.44275984  0.05823695
   0.58274662]
 [-0.35545545  0.04221355 -0.81353943 ... -0.73681659  0.10114052
   1.01259486]
 [-0.31492943  0.15609218 -0.74546616 ... -0.42618225  0.45762792
   0.83552733]]


In [4]:
#pip install tensorflow

# Use the Artificial Neuron Network (ANN) to train the model
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__

2022-08-10 17:55:00.093853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 17:55:00.093884: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.9.1'

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(seqcodings, neutralising, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
#Initializing the ANN
ann = tf.keras.models.Sequential()

2022-08-10 17:55:04.568980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-10 17:55:04.569336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 17:55:04.569417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-10 17:55:04.569471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-10 17:55:04.569523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [8]:
# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [9]:
# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [10]:
# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [11]:
# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
# Training the ANN on the Training set
X_train = np.array(X_train)
y_train = np.array(y_train)
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
75/75 [==============================] - 1s 1ms/step - loss: 0.4886 - accuracy: 0.7738
Epoch 2/100
75/75 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.7954
Epoch 3/100
75/75 [==============================] - 0s 1ms/step - loss: 0.4076 - accuracy: 0.7996
Epoch 4/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3920 - accuracy: 0.8029
Epoch 5/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3781 - accuracy: 0.8183
Epoch 6/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3692 - accuracy: 0.8225
Epoch 7/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3566 - accuracy: 0.8267
Epoch 8/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3466 - accuracy: 0.8363
Epoch 9/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3398 - accuracy: 0.8325
Epoch 10/100
75/75 [==============================] - 0s 1ms/step - loss: 0.3297 - accuracy: 0.8496
Epoch 11/

In [13]:
print ("len:", len(X_train), X_train)

len: 2400 [[ 1.91177638 -1.37914301 -0.01711973 ...  0.071948   -0.86682257
   1.04885515]
 [ 0.40828873  1.66285426  0.7683436  ... -0.14606984 -0.21671683
   0.77387369]
 [ 0.06129879  0.64096578  0.14537357 ...  0.17819591  0.71597356
   0.83455625]
 ...
 [ 0.15100135 -0.2044245   1.26322577 ...  0.16612532 -0.21241946
  -0.38830657]
 [-0.62988971 -2.59805259 -1.31028852 ...  0.01790867 -0.67751233
   0.78994512]
 [ 1.70490741 -1.89280508  0.17699117 ...  0.29894609  0.09576332
  -1.10152332]]


In [14]:
print ("len:", len(X_test), X_test)

len: 600 [[ 0.16025759  0.35031499 -0.308193   ... -0.82382215 -0.89080491
   0.84782852]
 [-0.2162426  -0.32633827  0.88079547 ... -1.40541159  0.15737595
  -0.05270441]
 [ 0.14510934 -3.11858044  0.92736741 ...  0.70652506 -0.38064735
  -0.36117366]
 ...
 [-0.31979006  0.34694813  0.75015523 ...  1.15863723  0.87475739
  -0.42168111]
 [-1.76719913 -1.32080877  0.123102   ... -1.58562114 -0.34928527
  -1.59780679]
 [-0.87844641 -0.4222064   0.37687927 ... -0.39090743 -0.81173503
   1.14218592]]


In [15]:
print ("len:", len(y_test), y_test)

len: 600 [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 

In [16]:
# Predicting the Test set results

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

#print ("len:", len(y_pred), y_pred)

#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

19/19 [==============================] - 0s 834us/step


In [17]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[402  73]
 [ 57  68]]


0.7833333333333333

In [22]:
#pip install icn3dpy
#pip install jupyterlab
#jupyterhub labextension install jupyterlab_3dmol

# show 3D structure

import icn3dpy

offset = int(MAXCNT * 0.8)
matchCnt = 0
totalCnt = len(y_pred)
bFound = False
for i in range(totalCnt):
    if(y_pred[i] == y_test[i]):
        matchCnt = matchCnt + 1

    if(pdbids[i+offset] != '' and bFound == False):
        bFound = True
        pdbid = pdbids[i+offset]
        seq = seqs[i+offset]
        print ("PDB ID:", pdbid, "seq:", seq)
        idStr = 'mmdbid=' + pdbid
        cmdStr = 'select :' + seq + ' | name test; color F00; style proteins sphere'
        view = icn3dpy.view(q=idStr,command=cmdStr)
        view
        
print("totalCnt:", totalCnt, "matchCnt:", matchCnt, "Matched ratio:", matchCnt/totalCnt)

PDB ID: 7M7B seq: QVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARRWWLRGAFDIWGQGTTVTVSS
totalCnt: 600 matchCnt: 470 Matched ratio: 0.7833333333333333


In [23]:
idStr = 'mmdbid=' + pdbid
cmdStr = 'select :' + seq + ' | name test; color F00; style proteins sphere'
print (idStr, cmdStr)

mmdbid=7M7B select :QVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARRWWLRGAFDIWGQGTTVTVSS | name test; color F00; style proteins sphere


In [24]:
view = icn3dpy.view(q=idStr,command=cmdStr)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the extension: 
 jupyter labextension install jupyterlab_3dmol